# 시퀀스
### 시퀀스데이터
- 시퀀스 데이터는 나열된 데이터를 의미
- 각 요소들이 동일한 속성을 띌 필요없고, 어떤 기준에 따라 정렬되지 않아도 된다.
- 살면서 마주치는 거의 모든 데이터들이 시퀀스 데이터

### RNN
- 어떤 문법적인 원리를 통해서가 아니고 수많은 글을 읽게 함으로써 다음에 올 문장을 예측
- 그 방식을 잘 처리하는 인공지능 중 하나가 RNN

나는 밥을 먹었다

In [1]:
sentence = "나는 밥을 먹었다"


source_sentence = "<start>" + sentence
target_sentence = sentence+"<end>"

print("source 문장 :", source_sentence)
print("Target 문장 :", target_sentence)

source 문장 : <start>나는 밥을 먹었다
Target 문장 : 나는 밥을 먹었다<end>


# 데이터 다듬기

### 데이터 다운로드

In [2]:
# 라이브러리
import os, re
import numpy as np
import tensorflow as tf

file_path = os.getenv("HOME") + '/aiffel/lyricist/data/shakespeare.txt'
with open(file_path, "r") as f: # 읽기전용('r')으로 file을 불러온다. file은 f로 칭한다.
    raw_corpus = f.read().splitlines() # file을 한줄씩 읽어오는데 .splitlines()로 종료문자 \n을 포함하지 않음.
    
print(raw_corpus[:9]) # 0~9 인덱스까지

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


- 원치않는문장:
[0,3,6] 번째 인덱스, [2,5,9] 번째 인덱스(공백 문장)
- 화자가 표기된 문장은 :로 끝나므로 : 기준으로 문장을 제외
- 공백인 문장은 길이를 검사하여 길이가 0이면 제외

In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: # sentence문장이 공백일때 제외
        continue
    if sentence[-1]==":": # sentence 문장의 맨끝이 ":"일때 제외
        continue
        
    if idx>9: # 우선 문장 10개만 확인
        break
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


### 몇 가지의 문제되는 케이스
- 1. Hi, my name is John. - 문장부호
- 2. First, open the first chapter. - 대소문자
- 3. He is a ten-year-old boy. - 특수문자

### 해결
- 1. 문장 부호양쪽에 공백추가
- 2. 모든 문자들을 소문자로 변환
- 3. 특수문자들은 모두 제거

In [13]:
# 정규표현식(Regex)를 이용한 필터링
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence


print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


- 모델의 입력이 되는 문장을 소스문장(source sentence), 정답역할을 하게 될 출력 문장을 타겟문장(Target sentence)라고 한다.
- source sen = X_train, Target sen = y_train

In [14]:
corpus = [] # 정제된 문장 넣는 빈 리스트

for sentence in raw_corpus:
    # 원치않는 문장 건너뛰기
    if len(sentence)==0: continue
    if sentence[-1]==":": continue
        
    # 정제하고 corpus에 넣기
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

이제 이 데이터들을 숫자로 변환해서 인공지능에게 줘야한다.
Tensorflow는 자연어 처리를 위한 모듈을 제공
정제된 데이터를 토큰화, 단어사전을 만들고, 데이터를 숫자로 변환까지 해주는것을 **벡터화**라고 하며, 숫자로 변환된 데이터를 **Tensor**라고 한다.

In [16]:
# 토큰화
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        # 7000단어를 기억할 수 있는 tokenizer
        # 이미 문장을 정제하였으니 filters가 필요없다.
        # 7000 단어에 포함되지 못한 단어는 '<unk>'로 변환
        num_words=7000,
        filters='',
        oov_token='<unk>'
    )
    # corpus를 이용하여 tokenizer 내부의 단어장 완성
    tokenizer.fit_on_texts(corpus)
    # tokenizer 이용하여 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)
    # 시퀀스 길이 일정하게
    # 시퀀스가 짧으면 문장 뒤에 패딩을 붙여 길이를 맞춰준다.
    # 문장 앞에 패딩을 붙여 길이를 맞추려면 padding='pre'사용
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f097a29dad0>


In [17]:
print(tensor[:3, :10])

[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]]


- tokenizer에 구축된 단어 사전의 인덱스

In [18]:
for idx in tokenizer.index_word:
    print(idx,":",tokenizer.index_word[idx]) # tokenizer변수에 저장되어 있는 단어 사전의 인덱스
    
    if idx >=10:break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


- 모든 인덱스가 2로 시작하는 이유를 알 수 있다.
- 사전에는 없지만 0 : < pad >가 된다.

In [19]:
# tensor에서 마지막 토큰을 잘라내서 소스문장 생성
# 마지막 토큰은 <end>가 아니고 <pad>일 가능성높음
src_input = tensor[:,:-1]
# tensor에서 <start>를 잘라내서 타겟문장 생성
tgt_input = tensor[:,1:]

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


데이터셋을 이미 tensor형태로 생성해 두었으므로, tf.data.Dataset객체를 생성한다.

In [20]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input)//BATCH_SIZE

# tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words+1

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset

dataset=tf.data.Dataset.from_tensor_slices((src_input, tgt_input)) # tensor --> tf.data.Dataset으로 변환
dataset=dataset.shuffle(BUFFER_SIZE) # 완벽한 셔플링을 위해서는 데이터셋의 전체 크기보다 크거나 같은 Buffersize필요
dataset=dataset.batch(BATCH_SIZE,drop_remainder=True) # 256개씩 묶고 나머지 제거
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

## 중요
- 1. 정규표현식을 이용한 corpus 생성
- 2. tf.keras.preprocessing.text.Tokenizer 이용한 corpus를 tensor로 변환
- 3. tf.data.Dataset.from_tensor_slices() 이용한 tensor를 tf.data.Dataset객체로 변환

## 이 모든 과정들을 데이터전처리라고 칭한다.

=========================================================================================================

## 실습 : 인공지능 학습

In [22]:
class TextGenerator(tf.keras.Model):
    def __init__(self,vocab_size, embedding_size,hidden_size):
        super().__init__()
        # embedding Layer
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)# 인덱스 값을 해당 인덱스 번째의 워드벡터로 바꿔준다. embedding_size : 단어가 추상적으로 표현되는 크기
        # 2개의 LSTM Layer
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True) # return_sequences : many to many
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True) 
        # 1개의 Dense Layer
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size=256
hidden_size=1024
model = TextGenerator(tokenizer.num_words+1, embedding_size, hidden_size)

In [23]:
for src_sample, tgt_sample in dataset.take(1):break # 1개의 배치를 가져온다. 256개의 문장데이터
    
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[ 4.65038784e-06, -2.24122632e-05, -3.65347252e-04, ...,
         -8.63523819e-05,  8.65471447e-05,  8.37919379e-06],
        [ 3.92112415e-04, -1.47646220e-04, -4.94217151e-04, ...,
         -4.46202408e-04,  1.27691150e-04,  4.90672188e-04],
        [ 5.89309202e-04, -5.40256951e-05, -3.20299412e-04, ...,
         -3.22111184e-04,  1.71948021e-04,  6.70864480e-04],
        ...,
        [-7.57864851e-04,  1.58081995e-03, -2.20646011e-03, ...,
         -4.80753195e-04,  1.50365615e-03, -2.33678264e-03],
        [-5.97875740e-04,  1.50429190e-03, -2.61203921e-03, ...,
         -6.58372010e-04,  1.70325220e-03, -2.49327393e-03],
        [-4.16337425e-04,  1.40979129e-03, -2.94919871e-03, ...,
         -8.22817499e-04,  1.88389444e-03, -2.61277473e-03]],

       [[ 4.65038784e-06, -2.24122632e-05, -3.65347252e-04, ...,
         -8.63523819e-05,  8.65471447e-05,  8.37919379e-06],
        [ 1.91560874e-04,  4.55135560e-05, -7.

In [24]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [25]:
# 혹시 미리 알고 싶다면 아래 문서를 참고
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
optimizer = tf.keras.optimizers.Adam() 
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
93/93 [==============================] - 58s 599ms/step - loss: 4.3851
Epoch 2/30
93/93 [==============================] - 64s 692ms/step - loss: 2.8146
Epoch 3/30
93/93 [==============================] - 65s 695ms/step - loss: 2.7154
Epoch 4/30
93/93 [==============================] - 65s 697ms/step - loss: 2.6167
Epoch 5/30
93/93 [==============================] - 65s 698ms/step - loss: 2.5328
Epoch 6/30
93/93 [==============================] - 65s 699ms/step - loss: 2.4911
Epoch 7/30
93/93 [==============================] - 65s 700ms/step - loss: 2.4396
Epoch 8/30
93/93 [==============================] - 65s 697ms/step - loss: 2.3811
Epoch 9/30
93/93 [==============================] - 65s 698ms/step - loss: 2.3250
Epoch 10/30
93/93 [==============================] - 65s 698ms/step - loss: 2.2689
Epoch 11/30
93/93 [==============================] - 65s 700ms/step - loss: 2.2235
Epoch 12/30
93/93 [==============================] - 65s 698ms/step - loss: 2.1832
Epoch 13/30
9

In [26]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [27]:
generate_text(model, tokenizer, init_sentence="<start> he") ## 4-6

'<start> he s a disease that made me <unk> , <end> '

- generat_text() 함수에서 init_sentence를 인자로 받고있다.